## Multi-Agent Systems
---

A simple implementation for multiagent system could be like this:

```mermaid
flowchart TD
    A[Manager Agent]
    A[Manager Agent] --> B[Code Interpreter Agent]
    A[Manager Agent] --> C[Web Search Agent]
    C[Web Search Agent] --> D[Web Search Tool]
    C[Web Search Agent] --> E[Visit Webpage Tool]
```

#### Solving complex task with a multi-agent hierarchy
---

**Problem Statement:** Cryptography exam is approaching and you have not made any notes to study at all. You want your agent to finalize the notes for you considering your syllabus.

In [1]:
##@ Lets build this !!
!pip install 'smolagents[litellm]' matplotlib geopandas shapely kaleido -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 18.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 77.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.6/104.6 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 567.3/567.3 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 65.6 MB/s eta 0:00:00:00:01


In [5]:
!git config --global credential.helper store

In [6]:
from huggingface_hub import notebook_login

notebook_login()

In [25]:
from smolagents import CodeAgent, HfApiModel, ToolCallingAgent, tool, GoogleSearchTool
import json
import os

@tool
def research_cryptography_topic(
    topic: str,
    subtopics: list = None,
    depth: str = "detailed"
) -> dict:
    """
    Research on provided cryptopgraphy topics from Course CSC316 from Tribhuvan University.

    Args:
        topic: The cryptography topic to research on
        subtopics: Specific subtopics to focus on 
        depth: How detailed the research should be

    Returns:
        Dictionary cointaining researched information
    """
    pass

@tool
def format_notes(
    content: dict,
    include_examples: bool = True,
    include_diagrams: bool = True
) -> str:
    """
    Format the provided content into well-structured notes.
    
    Args:
        content: The content to format
        include_examples: Whether to include examples
        include_diagrams: Whether to include diagrams
        
    Returns:
        Formatted notes in markdown
    """
    pass

@tool
def create_algo_visualization(
    algorithm_name: str,
    steps: list = None,
    visualization_type: str = "flowchart"
) -> str:
    """
    Create a text-based visualization of a cryptography algorithm.

    Args:
        algorithm_name: Name of the algorithm
        steps: The steps involved in the algorithm
        visualization_type: Type of visualization you want to see. (eg, flowchart, graphs etc.)

    Returns:
        Text-based visualization
    """
    pass

@tool
def generate_practice_questions(
    topic: str,
    difficulty: str = "medium",
    count: int = 3,
    include_solutions: bool = True
) -> str:
    """
    Generate practice questions for a cryptography topic.
    
    Args:
        topic: The topic to create questions for
        difficulty: The difficulty level
        count: Number of questions to create
        include_solutions: Whether to include solutions
        
    Returns:
        Practice questions with answers
    """
    pass

@tool
def save_notes_to_file(
    unit_number: int,
    unit_name: str,
    content: str
) -> str:
    """
    Save generated notes to a file.
    
    Args:
        unit_number: The unit number
        unit_name: Name of the unit
        content: Content to save
        
    Returns:
        Confirmation message
    """
    filename = f"unit_{unit_number}_{unit_name.replace(' ', '_').lower()}.md"
    with open(filename, "w") as f:
        f.write(content)
    return f"Notes for Unit {unit_number}: {unit_name} saved to {filename}"

In [35]:
model = HfApiModel(model_id="deepseek-ai/DeepSeek-R1")

In [14]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("SERPAPI_API_KEY")

In [16]:
import os

os.environ["SERPAPI_API_KEY"] = secret_value_0

# Access the variable
print(os.getenv("MY_VARIABLE"))  

In [36]:
## Creating specialized research agent
research_agent = ToolCallingAgent(
    model=model,
    tools=[research_cryptography_topic, GoogleSearchTool()],
    name="research_specialist",  
    description="Gathers detailed information on cryptography topics from reliable sources",  
)

In [37]:
## Creating specialized note formatting agent 
formatter_agent = ToolCallingAgent(
    model=model,
    tools=[format_notes, create_algo_visualization],
    name="note_formatter", 
    description="Formats cryptography unit-wise information into structured notes with visualizations",
    max_steps=5  
)

## Creating practice questions generating agent
question_agent = ToolCallingAgent(
    model=model,
    tools=[generate_practice_questions],
    name="question_generator",
    description="Generates practice questions for cryptography topics",
)

manager_agent = CodeAgent(
    model=model,
    tools=[save_notes_to_file],
    managed_agents=[formatter_agent, research_agent, question_agent],  
    name="notes_manager",
    additional_authorized_imports=["markdown", "requests", "bs4", "pandas", "numpy", "final_answer"],
    description="Orchestrates note generation workflow",
    max_steps=20
)

In [38]:
syllabus = """
Course Description : The course introduces the underlying the principles and design of cryptosystems. The course covers the basics concepts of cryptography including: traditional ciphers, block ciphers, stream ciphers, public and private key cryptosystems. The course also includes the theory of hash functions, authentication systems, network security protocols and malicious logic.

Course Objective : The objectives of this course are to familiarize the students with cryptography and its applications. The students will be able to develop basic understanding of cryptographic mechanisms.

Course Contents:
Unit 1. Introduction and Classical Ciphers.
1.4. Security: Computer Security, Information Security, Network Security, CIA Triad, Cryptography, Cryptosystem, Cryptanalysis, Security Threats and Attacks, Security Services, Security Mechanisms
1.5. Classical Cryptosystems: Substitution Techniques: Ceasar, Monoalphabetic, Playfair, Hill, Polyalphabetic ciphers, One-time pad Transposition Techniques: Rail Fence Cipher
1.6. Modern Ciphers: Block vs. Stream Ciphers, Symmetric vs. Asymmetric Ciphers

Unit 2. Symmetric Ciphers.
2.4. Feistel Cipher Structure, Substitution Permutation Network (SPN)
2.5. Data Encryption Standards (DES), Double DES, Triple DES
2.6. Finite Fields: Groups Rings, Fields, Modular Arithmetic, Euclidean Algorithm, Galois Fields (GF(p) & GF(2n)), Polynomial Arithmetic
2.7. International Data Encryption Standard (IDEA)
2.8. Advanced Encryption Standards (AES) Cipher
2.9. Modes of Block Cipher Encryptions (Electronic Code Book, Cipher Block Chaining, Cipher Feedback Mode, Output Feedback Mode, Counter Mode)

Unit 3. Asymmetric Ciphers.
3.4. Number Theory: Prime Numbers, Fermat's Theorem, Euler's Theorem, Primility Testing, Miller-Rabin Algorithm, Extended Euclidean Theorem, Discrete Logarithms
3.5. Public Key Cryptosystems, Applications of Public Key Cryptosystems
3.6. Distribution of public key, Distribution of secret key by using public key cryptography, Diffie-Helman Key Exchange, Man-in-the-Middle Attack
3.7. RSA Algorithm
3.8. Elgamal Cryptographic System

Unit 4. Cryptographic Hash Functions and Digital Signatures.
4.4. Message Authentication, Message Authentication Functions, Message Authentication Codes
4.5. Hash Functions, Properties of Hash functions, Applications of Hash Functions
4.6. Message Digests: MD4 and MD5
4.7. Secure Hash Algorithms: SHA-1 and SHA-2
4.8. Digital Signatures: Direct Digital Signatures, Arbitrated Digital Signature
4.9. Digital Signature Standard: The DSS Approach, Digital Signature Algorithm
4.10. Digital Signature Standard: The RSA Approach

Unit 5. Authentication.
5.4. Authentication System,
5.5. Password Based Authentication, Dictionary Attacks,
5.6. Challenge Response System,
5.7. Biometric System
5.8. Needham-Schroeder Scheme, Kerberos Protocol

Unit 6. Network Security and Public Key Infrastructure6 Hrs.
6.1. Overview of Network Security
6.2. Digital Certificates and X.509 certificates, Certificate Life Cycle Management
6.3. PKI trust models, PKIX
6.4. Email Security: Pretty Good Privacy (PGP)
6.5. Secure Socket Layer (SSL) and Transport Layer Security (TLS)
6.6. IP Security (IPSec)
6.7. Firewalls and their types

Unit 7. Malicious Logic.
7.1. Malicious Logic, Types of Malicious Logic: Virus, Worm, Trojan Horse, Zombies, Denial of Service Attacks,
7.2. Intrusion, Intruders and their types, Intrusion Detection System
"""

In [39]:

result = manager_agent.run(f"""
I need comprehensive notes for my upcoming cryptography exam. I haven't made any notes yet and need a complete study guide based on this syllabus:

{syllabus}

Please follow this structured workflow:
1. For each unit in the syllabus:
   a. Have the research specialist gather detailed information on all topics in that unit
   b. Have the note formatter create well-structured notes with examples and visualizations
   c. Have the question generator create 5 practice questions (2 easy, 2 medium, 1 hard)
   d. Save the combined notes and questions as a separate file for each unit

2. After all units are completed, create a master index document that links to all unit files.

The notes should be comprehensive, clear, and well-organized to help me prepare effectively for the exam.
""")

╭──────────────────────────────────────────── New run - notes_manager ────────────────────────────────────────────╮
│                                                                                                                 │
│ I need comprehensive notes for my upcoming cryptography exam. I haven't made any notes yet and need a complete  │
│ study guide based on this syllabus:                                                                             │
│                                                                                                                 │
│                                                                                                                 │
│ Course Description : The course introduces the underlying the principles and design of cryptosystems. The       │
│ course covers the basics concepts of cryptography including: traditional ciphers, block ciphers, stream         │
│ ciphers, public and private key cryptosystems. The course also includes the theory of hash functions,           │
│ authentication systems, network security protocols and malicious logic.                                         │
│                                                                                                                 │
│ Course Objective : The objectives of this course are to familiarize the students with cryptography and its      │
│ applications. The students will be able to develop basic understanding of cryptographic mechanisms.             │
│                                                                                                                 │
│ Course Contents:                                                                                                │
│ Unit 1. Introduction and Classical Ciphers.                                                                     │
│ 1.4. Security: Computer Security, Information Security, Network Security, CIA Triad, Cryptography,              │
│ Cryptosystem, Cryptanalysis, Security Threats and Attacks, Security Services, Security Mechanisms               │
│ 1.5. Classical Cryptosystems: Substitution Techniques: Ceasar, Monoalphabetic, Playfair, Hill, Polyalphabetic   │
│ ciphers, One-time pad Transposition Techniques: Rail Fence Cipher                                               │
│ 1.6. Modern Ciphers: Block vs. Stream Ciphers, Symmetric vs. Asymmetric Ciphers                                 │
│                                                                                                                 │
│ Unit 2. Symmetric Ciphers.                                                                                      │
│ 2.4. Feistel Cipher Structure, Substitution Permutation Network (SPN)                                           │
│ 2.5. Data Encryption Standards (DES), Double DES, Triple DES                                                    │
│ 2.6. Finite Fields: Groups Rings, Fields, Modular Arithmetic, Euclidean Algorithm, Galois Fields (GF(p) &       │
│ GF(2n)), Polynomial Arithmetic                                                                                  │
│ 2.7. International Data Encryption Standard (IDEA)                                                              │
│ 2.8. Advanced Encryption Standards (AES) Cipher                                                                 │
│ 2.9. Modes of Block Cipher Encryptions (Electronic Code Book, Cipher Block Chaining, Cipher Feedback Mode,      │
│ Output Feedback Mode, Counter Mode)                                                                             │
│                                                                                                                 │
│ Unit 3. Asymmetric Ciphers.                                                                                     │
│ 3.4. Number Theory: Prime Numbers, Fermat's Theorem, Euler's Theorem, Primility Testing, Miller-Rabin           │
│ Algorithm, Extended Euclidean Theorem, Discrete Logari

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Error in generating model output:
402 Client Error: Payment Required for url: 
https://router.huggingface.co/hf-inference/models/deepseek-ai/DeepSeek-R1/v1/chat/completions (Request ID: 
Root=1-67d1dcc5-3d197a283bcf60213a73c799;94bdd536-e365-4de9-809b-130a0a37e1cc)

You have exceeded your monthly included credits for Inference Providers. Subscribe to PRO to get 20x more monthly 
included credits.

[Step 1: Duration 0.07 seconds]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 20 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Error in generating model output:
402 Client Error: Payment Required for url: 
https://router.huggingface.co/hf-inference/models/deepseek-ai/DeepSeek-R1/v1/chat/completions (Request ID: 
Root=1-67d1dcc7-3d9cc1a32caafd777711feb4;63db0264-19c6-4f5a-a4c4-31c6abd507bb)

You have exceeded your monthly included credits for Inference Providers. Subscribe to PRO to get 20x more monthly 
included credits.

[Step 20: Duration 0.05 seconds]

Reached max steps.

[Step 21: Duration 0.11 seconds]

well I ran out my monthly included credits for inference providers in HF... 🤷‍♂️